In [191]:
import pandas as pd
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from datetime import datetime,date,timedelta
import matplotlib.pyplot  as plt

In [199]:
#The Module will prompt you for your CoinMarketCap API Key 
#...it will then use your Key to query and pull the top 500 Cryptos continaing current to 90 day prior price change data

crypto_data = []
api_key = input('Enter your coinmarketcap API key: ')
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'500',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '{}'.format(api_key),
}

session = Session()
session.headers.update(headers)

try:
    response = session.get(url, params=parameters)
    data = json.loads(response.text)
    df = pd.json_normalize(data['data'])

except (ConnectionError, Timeout, TooManyRedirects) as e:
    print ('Error')


Enter your coinmarketcap API key: b2da9a8a-d2cc-44df-85e6-2b38dc6f150a


In [205]:
#This module defines the functions used to format and plot the price change graphs as well as the drop down widget

def f(Crypto):
    current_price = df[df['name'] == Crypto]['quote.USD.price'].item()
    price_1h = current_price + (current_price * df[df['name'] == Crypto]['quote.USD.percent_change_1h'].item()/100)
    price_24h = current_price + (current_price * df[df['name'] == Crypto]['quote.USD.percent_change_24h'].item()/100)
    price_7d = current_price + (current_price * df[df['name'] == Crypto]['quote.USD.percent_change_7d'].item()/100)
    price_30d = current_price + (current_price * df[df['name'] == Crypto]['quote.USD.percent_change_30d'].item()/100)
    price_60d = current_price + (current_price * df[df['name'] == Crypto]['quote.USD.percent_change_60d'].item()/100)
    price_90d = current_price + (current_price * df[df['name'] == Crypto]['quote.USD.percent_change_90d'].item()/100)
    
    date_1 = (datetime.now() - timedelta(1)).strftime("%y%m%d")
    date_7 = (datetime.now() - timedelta(7)).strftime("%y%m%d")
    date_30 = (datetime.now() - timedelta(30)).strftime("%y%m%d")
    date_60 = (datetime.now() - timedelta(60)).strftime("%y%m%d")
    date_90 = (datetime.now() - timedelta(90)).strftime("%y%m%d")
    
    y=[price_90d,price_60d,price_30d,price_7d,price_24h,current_price]
    x=[date_90,date_60,date_30,date_7,date_1, datetime.now().strftime("%y%m%d")]
    plt.plot(x,y)
    plt.title('{}: 90 Day Price VS. Current'.format(Crypto))
    plt.xlabel('Date (YYMMDD)')
    plt.ylabel('Price ($USD)')
    plt.show()
    
    print('Current Value (USD): ${}'.format(current_price))    
    print('1 Hour Value (USD): ${}'.format(price_1h))
    print('24 Hour Value (USD): ${}'.format(price_24h))
    print('7 Day Value (USD): ${}'.format(price_7d))
    print('30 Day Value (USD): ${}'.format(price_30d))
    print('60 Day Value (USD): ${}'.format(price_60d))
    print('90 Day Value (USD): ${}'.format(price_90d))
    
def graph_cryptos():
    names=set(df['name'])
    names=sorted(names)
    interact(f, Crypto=names)

In [206]:
#This module builds the visualization

graph_cryptos()

interactive(children=(Dropdown(description='Crypto', options=('0x', '1eco', '1inch Network', 'ABBC Coin', 'AIO…